# Implementation of the chatbot as a backend structure

In this file, you can find the implementation of the backend architecture of my software project.

The backend structure of my project is designed using the Python programming language. There are many reasons that I chose Python as the backend language, mainly because of the easiness it provides while developing machine learning models and the library support of it(mainly for machine learning and artificial intelligence)

## Importing the libraries needed

In addition to the language, I also used some libraries that is built for Python language to build the backend. Here are the libraries I used:

### Random

In [1]:
import random

I used the random library in order to randomize the x and y input arrays on the training and set data.

### JSON

In [2]:
import json

I used the JSON library, so that I could use my training dataset(intents.json file), to train the deep learning model that I created later on.

### NumPy

In [3]:
import numpy as np

I used the NumPy library in order to do some scientific computing and some matrix calculations in my project.

### NLTK

In [4]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alicagatay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alicagatay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/alicagatay/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

I used the NLTK package in order to be able to work with texts in the machine learning model that I developed later.

### Tensorflow

In [5]:
import tensorflow as tf

I used the Tensorflow library in order to build the deep learning model that I used while developing the machine learning model

### Scikit-learn

In [6]:
from sklearn.model_selection import train_test_split

I used the Scikit-learn library in order to split the dataset into training and testing data.

### OS

In [7]:
import os

I used the os library so that I can open diferent directories when I need to(for example, to open the training dataset inside another folder, to open the model inside another directory, etc.)

## Preprocessing the dataset

The first thing I did before creating the machine learning model was to preprocess the dataset so that I can use it to train the model. In order to accomplish this, I first loaded the dataset inside the intents.json file.

In [8]:
chatbotData = os.path.relpath('..//data')
chatbotIntents = json.loads(open(chatbotData + '/intents.json').read())
exerciseData = json.loads(open(chatbotData + '/list_of_all_exercises.json').read())

After loading the dataset from intents.json file, I separated the the texts into 3 different arrays; words, documents and classes. I also created an array called ignoreLetters, in order to ignore all the punctuation marks from the texts.

In [9]:
chatbotWords = []
chatbotClasses = []
chatbotDocuments = []
ignoreLetters = ['?', '!', '.', ',', "'", '"']
for intent in chatbotIntents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        chatbotWords.extend(word_list)
        chatbotDocuments.append((word_list, intent['tag']))
        if intent['tag'] not in chatbotClasses:
            chatbotClasses.append(intent['tag'])

After creating these arrays, I lemmatised the words in the words array. I also used the ignoreLetters array to ignore some of the words that I used in my training data.

In [10]:
chatbotLemmatizer = nltk.stem.WordNetLemmatizer()

for word in chatbotWords:
    if word not in ignoreLetters:
        word = chatbotLemmatizer.lemmatize(word)

After lemmatising the words, I sorted the arrays words and classes in order to make sure that the words in the words array are in the same order as the words in the classes array.

In [11]:
chatbotWords = sorted(set(chatbotWords))
chatbotClasses = sorted(set(chatbotClasses))

After the conversion, I used the words array to create a bag of words for every document out there and added those bag of words into an array that I later used to create the training and testing datasets.

In [12]:
chatbotData = []
output_empty = [0] * len(chatbotClasses)

for document in chatbotDocuments:
    bag = []
    wordPatterns = document[0]


    for word in wordPatterns:
        word = chatbotLemmatizer.lemmatize(word.lower())

    for word in chatbotWords:
        if word in wordPatterns:
            bag.append(1)
        else:
            bag.append(0)

    output_row = list(output_empty)
    output_row[chatbotClasses.index(document[1])] = 1
    chatbotData.append([bag, output_row])

I then randomised the contents of the data array so that the order of the bag of words is randomised.

In [13]:
random.shuffle(chatbotData)

I then splitted the data array into training and testing arrays.

In [14]:
training_data, testing_data = train_test_split(chatbotData, test_size=0.1, random_state=25)


After that, just before creating the neural network model, I separated the training dataset into two different arrays called train_x and train_y.

In [15]:
training_data = np.array(training_data)
train_x = list(training_data[:, 0])
train_y = list(training_data[:, 1])

/var/folders/_9/tgxc64p92ks65f7thjwm8fwc0000gn/T/ipykernel_65655/2759127376.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)


Same for the testing dataset.

In [16]:
testing_data = np.array(testing_data)
test_x = list(testing_data[:, 0])
test_y = list(testing_data[:, 1])

/var/folders/_9/tgxc64p92ks65f7thjwm8fwc0000gn/T/ipykernel_65655/3973432232.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  testing_data = np.array(testing_data)


## Creating the neural network model

In [17]:
chatbotModel = tf.keras.models.Sequential()
chatbotModel.add(tf.keras.layers.Dense(len(train_x[0]), input_shape=(len(train_x[0]),), activation='relu'))
chatbotModel.add(tf.keras.layers.Dropout(0.5))
chatbotModel.add(tf.keras.layers.Dense(len(train_x[0])/2*3, activation='relu'))
chatbotModel.add(tf.keras.layers.Dropout(0.5))
chatbotModel.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))


adam = tf.keras.optimizers.Adam(learning_rate=0.01)
chatbotModel.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model_train = chatbotModel.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=len(train_x[0]), verbose=1)

2022-04-07 16:27:58.587763: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-07 16:27:58.587872: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-07 16:27:58.671046: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/100
5/5 [==============================] - 0s 13ms/step - loss: 2.9889 - accuracy: 0.0691
Epoch 2/100
1/5 [=====>........................] - ETA: 0s - loss: 2.7966 - accuracy: 0.1493

2022-04-07 16:27:58.847968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 9ms/step - loss: 2.7359 - accuracy: 0.1600
Epoch 3/100
5/5 [==============================] - 0s 9ms/step - loss: 2.5369 - accuracy: 0.1709
Epoch 4/100
5/5 [==============================] - 0s 9ms/step - loss: 2.2252 - accuracy: 0.2873
Epoch 5/100
5/5 [==============================] - 0s 9ms/step - loss: 1.9577 - accuracy: 0.3745
Epoch 6/100
5/5 [==============================] - 0s 9ms/step - loss: 1.5599 - accuracy: 0.5527
Epoch 7/100
5/5 [==============================] - 0s 9ms/step - loss: 1.3434 - accuracy: 0.6000
Epoch 8/100
5/5 [==============================] - 0s 9ms/step - loss: 1.1842 - accuracy: 0.6145
Epoch 9/100
5/5 [==============================] - 0s 9ms/step - loss: 0.8687 - accuracy: 0.7600
Epoch 10/100
5/5 [==============================] - 0s 9ms/step - loss: 0.7171 - accuracy: 0.7964
Epoch 11/100
5/5 [==============================] - 0s 9ms/step - loss: 0.6840 - accuracy: 0.7891
Epoch 12/100
5/5 [======================

In [18]:
model_test = chatbotModel.evaluate(np.array(test_x), np.array(test_y), verbose=1, batch_size=len(test_x[0]))

1/1 [==============================] - 0s 90ms/step - loss: 0.2500 - accuracy: 0.9677


2022-04-07 16:28:03.698199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [19]:
models = os.path.relpath('..//models')
chatbotModel.save(models + '/chatbot_model.h5', model_train)
print("Done")

Done


After defining those variables, I started to implement some functions that I will use in the chatbot.

The first one is called "clean_up_sentence". This function is used to clean up the user's input so that it can be converted to a computable state for the model.

In [20]:
def clean_up_sentence(sentence):
    sentenceWords = nltk.word_tokenize(sentence)

    for word in sentenceWords:
        word = chatbotLemmatizer.lemmatize(word.lower())
        
    return sentenceWords

The second one is called "bag_of_words". This function is used to create a bag of words for the user's input.

In [21]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(chatbotWords)
    for w in sentence_words:
        for i, word in enumerate(chatbotWords):
            if word == w:
                bag[i] = 1
    return np.array(bag)


The other function that I created is called "predict_class". This function is used to predict a valid response for the user's input.

In [22]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = chatbotModel.predict(np.array([bow]))[0]
    error_threshold = 0.50
    results = [[i, r] for i, r in enumerate(res) if r > error_threshold]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in results:
        return_list.append({'intent': chatbotClasses[r[0]], 'probability': str(r[1])})
    return return_list

The other function that I created is called "get_response". This function is used to get a response for the user's input.

In [23]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result

In [24]:
waist_exercises_gym = []
back_exercises_gym = []
cardio_exercises_gym = []
chest_exercises_gym = []
lower_arm_exercises_gym = []
lower_leg_exercises_gym = []
neck_exercises_gym = []
shoulder_exercises_gym = []
upper_arm_exercises_gym = []
upper_leg_exercises_gym = []


waist_exercises_calisthenics = []
back_exercises_calisthenics = []
cardio_exercises_calisthenics = []
chest_exercises_calisthenics = []
lower_arm_exercises_calisthenics = []
lower_leg_exercises_calisthenics = []
neck_exercises_calisthenics = []
shoulder_exercises_calisthenics = []
upper_arm_exercises_calisthenics = []
upper_leg_exercises_calisthenics = []

for exercise in exerciseData:
    if exercise['bodyPart'] == 'back':
        if exercise['equipment'] == 'body weight':
            back_exercises_calisthenics.append(exercise)
        else:
            back_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'cardio':
        if exercise['equipment'] == 'body weight':
            cardio_exercises_calisthenics.append(exercise)
        else:
            cardio_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'chest':
        if exercise['equipment'] == 'body weight':
            chest_exercises_calisthenics.append(exercise)
        else:
            chest_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'lower arms':
        if exercise['equipment'] == 'body weight':
            lower_arm_exercises_calisthenics.append(exercise)
        else:
            lower_arm_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'lower legs':
        if exercise['equipment'] == 'body weight':
            lower_leg_exercises_calisthenics.append(exercise)
        else:
            lower_leg_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'neck':
        if exercise['equipment'] == 'body weight':
            neck_exercises_calisthenics.append(exercise)
        else:
            neck_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'shoulders':
        if exercise['equipment'] == 'body weight':
            shoulder_exercises_calisthenics.append(exercise)
        else:
            shoulder_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'waist':
        if exercise['equipment'] == 'body weight':
            waist_exercises_calisthenics.append(exercise)
        else:
            waist_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'upper arms':
        if exercise['equipment'] == 'body weight':
            upper_arm_exercises_calisthenics.append(exercise)
        else:
            upper_arm_exercises_gym.append(exercise)
    if exercise['bodyPart'] == 'upper legs':
        if exercise['equipment'] == 'body weight':
            upper_leg_exercises_calisthenics.append(exercise)
        else:
            upper_leg_exercises_gym.append(exercise)

The last function that I created is called "run_chatbot". This function lets me run an instance of the chatbot after every call of the function.

In [25]:
def run_chatbot(message):
    ints = predict_class(message)
    try:
        response = get_response(ints, chatbotIntents)

        if response == 'back_gym':
            response = random.choice(back_exercises_gym)
        if response == 'cardio_gym':
            response = random.choice(cardio_exercises_gym)
        if response == 'chest_gym':
            response = random.choice(chest_exercises_gym)
        if response == 'lower_arms_gym':
            response = random.choice(lower_arm_exercises_gym)
        if response == 'lower_legs_gym':
            response = random.choice(lower_leg_exercises_gym)
        if response == 'neck_gym':
            response = random.choice(neck_exercises_gym)
        if response == 'shoulders_gym':
            response = random.choice(shoulder_exercises_gym)
        if response == 'upper_arms_gym':
            response = random.choice(upper_arm_exercises_gym)
        if response == 'upper_legs_gym':
            response = random.choice(upper_leg_exercises_gym)
        if response == 'waist_gym':
            response = random.choice(waist_exercises_gym)
        if response == 'back_calisthenics':
            response = random.choice(back_exercises_calisthenics)
        if response == 'cardio_calisthenics':
            response = random.choice(cardio_exercises_calisthenics)
        if response == 'chest_calisthenics':
            response = random.choice(chest_exercises_calisthenics)
        if response == 'lower_arms_calisthenics':
            response = random.choice(lower_arm_exercises_calisthenics)
        if response == 'lower_legs_calisthenics':
            response = random.choice(lower_leg_exercises_calisthenics)
        if response == 'neck_calisthenics':
            response = random.choice(neck_exercises_calisthenics)
        if response == 'shoulders_calisthenics':
            response = random.choice(shoulder_exercises_calisthenics)
        if response == 'upper_arms_calisthenics':
            response = random.choice(upper_arm_exercises_calisthenics)
        if response == 'upper_legs_calisthenics':
            response = random.choice(upper_leg_exercises_calisthenics)
        if response == 'waist_calisthenics':
            response = random.choice(waist_exercises_calisthenics)
    except:
        response = {
        "bodyPart": "N/A",
        "equipment": "N/A",
        "gifUrl": "https://www.iconpacks.net/icons/2/free-sad-face-icon-2691-thumb.png",
        "id": "N/A",
        "name": "N/A",
        "target": "N/A"
    }
    return response

After defining all those functions, I implemented a simple server using the Flask framework, so that I can run the chatbot on my local machine and I can connect the backend of the project to the frontend framework.

In [26]:
from flask import Flask, request

app = Flask(__name__)


@app.route('/', methods=['GET'])
def run_bot():
    message = request.args.get('msg')
    response = run_chatbot(message)
    return response



app.run(host='localhost', port=3000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:3000/ (Press CTRL+C to quit)
2022-04-07 16:28:25.185237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
127.0.0.1 - - [07/Apr/2022 16:28:25] "GET /?msg=can%20you%20advise%20me%20a%20lower%20arm%20workout%20for%20the%20home%20please HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 16:28:42] "GET /?msg=lower%20arm%20wrkout%20gym%20pls HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 16:28:44] "GET /?msg=lower%20arm%20wrkout%20gym%20pls HTTP/1.1" 200 -
